<a href="https://colab.research.google.com/github/mr-cri-spy/Generative-AI-Projects/blob/main/dream11_prediction_using_deeplearning_%26ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [1]:
!pip install pandas numpy tensorflow keras gym cricketpy beautifulsoup4 requests


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00
  Created wheel for cricketpy: filename=cricketpy-1.2.6-py3-none-any.whl size=3474 sha256=b499a9242ab59a994210fc6ae490eb4ac8bb5ab3e0a69eb00189f08a421749c1
  Stored in directory: /root/.cache/pip/wheels/94/7c/70/a9d4381541cde3a0661eeb53e4ca9ba34f636d67e274604d45
  Created wheel for criapi: filename=criapi-1.1.0-py3-none-any.whl size=2924 sha256=d25575cd93dfb427ae5444eed16bc1852637c0e59460ce91e47a5920cb3d37d9
  Stored in directory: /root/.cache/pip/wheels/68/93/fa/a879b2da695876067aed162cb8a3a742f142579b6ee5cc3c27
Successfully built cricketpy criapi


Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import gym
import requests
from bs4 import BeautifulSoup


Scrape Real-Time Cricket Data

In [3]:
def get_live_data():
    url = "https://www.espncricinfo.com/live-cricket-score"  # Replace with correct API/website
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Example: Extract player stats
    players = soup.find_all("div", class_="player-info")
    data = []
    for player in players[:11]:  # Get top 11 players
        name = player.find("span", class_="player-name").text
        points = np.random.randint(20, 100)  # Mock Dream11 points (replace with real data)
        data.append([name, points])

    return pd.DataFrame(data, columns=["Player", "Points"])

# Get live data
live_data = get_live_data()
print(live_data)


Empty DataFrame
Columns: [Player, Points]
Index: []


Preprocess Data for LSTM

In [ ]:
def preprocess_data(df):
    df["Points"] = df["Points"].astype(float)  # Ensure numeric data
    X = df["Points"].values.reshape(-1, 1)  # Convert to NumPy array
    return X

X_train = preprocess_data(live_data)



Build LSTM Model

In [ ]:
model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(X_train.shape[0], 1)),
    Dropout(0.2),
    LSTM(50, activation="relu"),
    Dropout(0.2),
    Dense(1)  # Predict Dream11 points
])

model.compile(optimizer="adam", loss="mse")
model.summary()


Train LSTM Model

In [ ]:
# Simulating data for training
X_train = np.array([X_train for _ in range(10)])  # Mock time-series
y_train = np.random.randint(20, 100, size=(10,))  # Random points

model.fit(X_train, y_train, epochs=10, batch_size=2)


Reinforcement Learning for Team Selection

In [ ]:
import gym
from gym import spaces

class Dream11Env(gym.Env):
    def __init__(self):
        super(Dream11Env, self).__init__()
        self.action_space = spaces.Discrete(11)  # Select 11 players
        self.observation_space = spaces.Box(low=0, high=100, shape=(11,))  # Player performance

    def reset(self):
        self.state = np.random.randint(20, 100, size=(11,))  # Mock performance
        return self.state

    def step(self, action):
        reward = self.state[action]  # Reward based on selected player
        done = True
        return self.state, reward, done, {}

env = Dream11Env()


Train RL Model

In [4]:
import random

def train_rl(env, episodes=100):
    total_rewards = []
    for ep in range(episodes):
        state = env.reset()
        action = random.randint(0, 10)  # Randomly select a player
        _, reward, _, _ = env.step(action)
        total_rewards.append(reward)
        print(f"Episode {ep+1}: Reward = {reward}")

train_rl(env)


Final Step: Predict Best Players

In [ ]:
def select_best_team():
    live_data = get_live_data()
    X_live = preprocess_data(live_data)
    predictions = model.predict(np.array([X_live for _ in range(10)]))
    live_data["Predicted Points"] = predictions.mean(axis=0)
    best_team = live_data.sort_values(by="Predicted Points", ascending=False).head(11)
    print("🏏 Best Dream11 Team:")
    print(best_team)

select_best_team()